# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside `cs231n/classifiers/softmax.py`.


In [5]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.330618
sanity check: 2.302585


**Inline Question 1**

Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

$\color{blue}{\textit Your Answer:}$ *Fill this in* 

expectation for fj term is 0.5 since its randomized between 0 and 1, there are 10 terms, and a random one is denoted as correct, so we have a fraction equal to about 0.5/5



In [9]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: -0.020774 analytic: -0.020774, relative error: 4.638633e-09
numerical: 0.235284 analytic: 0.235284, relative error: 2.270673e-07
numerical: 1.417116 analytic: 1.417116, relative error: 1.192873e-08
numerical: 2.052060 analytic: 2.052060, relative error: 1.511464e-08
numerical: 1.118652 analytic: 1.118652, relative error: 9.134620e-09
numerical: 0.469407 analytic: 0.469407, relative error: 2.564356e-08
numerical: 0.166864 analytic: 0.166864, relative error: 3.797564e-08
numerical: -1.449454 analytic: -1.449454, relative error: 1.005428e-08
numerical: -1.544220 analytic: -1.544220, relative error: 9.592847e-09
numerical: 4.108431 analytic: 4.108431, relative error: 9.380143e-09
numerical: -1.828505 analytic: -1.824794, relative error: 1.015764e-03
numerical: -1.480965 analytic: -1.479856, relative error: 3.747962e-04
numerical: 0.827152 analytic: 0.821735, relative error: 3.284983e-03
numerical: 2.805477 analytic: 2.800045, relative error: 9.691794e-04
numerical: -2.528247 ana

In [22]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.330618e+00 computed in 0.104341s
vectorized loss: 2.330618e+00 computed in 0.005243s
Loss difference: 0.000000
Gradient difference: 0.000000


In [25]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.

from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################

# Provided as a reference. You may or may not want to change these hyperparameters
learning_rates = [1e-7, 5e-7]
reg_strengths = [2.5e4, 5e4]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

learning_rate_splits = np.linspace(learning_rates[0],learning_rates[1],num=15)
reg_splits = np.linspace(reg_strengths[0],reg_strengths[1],num=15)

for i in range(10):
    curr_lr = learning_rate_splits[i]
    for j in range(10):
        softmax = Softmax()
        curr_reg = reg_splits[j]
        loss_hist = softmax.train(X_train, y_train, learning_rate=1e-7, reg=2.5e4, num_iters=1000, verbose=True)
        y_train_pred = softmax.predict(X_train)
        y_val_pred = softmax.predict(X_val)
        val_acc = np.mean(y_val == y_val_pred)
        train_acc = np.mean(y_train == y_train_pred)
        results[(curr_lr,curr_reg)] = (train_acc,val_acc)
        if val_acc > best_val:
            best_val = val_acc
            best_softmax = softmax


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

iteration 0 / 1000: loss 763.460194
iteration 100 / 1000: loss 461.442525
iteration 200 / 1000: loss 279.958318
iteration 300 / 1000: loss 169.942199
iteration 400 / 1000: loss 103.665732
iteration 500 / 1000: loss 63.570493
iteration 600 / 1000: loss 39.255742
iteration 700 / 1000: loss 24.587635
iteration 800 / 1000: loss 15.725943
iteration 900 / 1000: loss 10.302083
iteration 0 / 1000: loss 764.630485
iteration 100 / 1000: loss 462.132851
iteration 200 / 1000: loss 280.416938
iteration 300 / 1000: loss 170.224285
iteration 400 / 1000: loss 103.978033
iteration 500 / 1000: loss 63.593518
iteration 600 / 1000: loss 39.396060
iteration 700 / 1000: loss 24.675246
iteration 800 / 1000: loss 15.810858
iteration 900 / 1000: loss 10.337385
iteration 0 / 1000: loss 774.993167
iteration 100 / 1000: loss 468.137690
iteration 200 / 1000: loss 284.015811
iteration 300 / 1000: loss 172.519581
iteration 400 / 1000: loss 105.239827
iteration 500 / 1000: loss 64.464936
iteration 600 / 1000: loss 39

iteration 0 / 1000: loss 773.082835
iteration 100 / 1000: loss 467.092210
iteration 200 / 1000: loss 282.926301
iteration 300 / 1000: loss 171.847117
iteration 400 / 1000: loss 104.767205
iteration 500 / 1000: loss 64.309754
iteration 600 / 1000: loss 39.766705
iteration 700 / 1000: loss 24.922822
iteration 800 / 1000: loss 15.788032
iteration 900 / 1000: loss 10.495125
iteration 0 / 1000: loss 766.072944
iteration 100 / 1000: loss 463.483109
iteration 200 / 1000: loss 281.257793
iteration 300 / 1000: loss 170.857962
iteration 400 / 1000: loss 104.143685
iteration 500 / 1000: loss 63.831450
iteration 600 / 1000: loss 39.469996
iteration 700 / 1000: loss 24.791710
iteration 800 / 1000: loss 15.726329
iteration 900 / 1000: loss 10.356230
iteration 0 / 1000: loss 769.843734
iteration 100 / 1000: loss 465.467208
iteration 200 / 1000: loss 282.353864
iteration 300 / 1000: loss 171.516132
iteration 400 / 1000: loss 104.599409
iteration 500 / 1000: loss 64.052997
iteration 600 / 1000: loss 39

iteration 0 / 1000: loss 771.202106
iteration 100 / 1000: loss 466.885142
iteration 200 / 1000: loss 283.001200
iteration 300 / 1000: loss 172.027349
iteration 400 / 1000: loss 104.937837
iteration 500 / 1000: loss 64.237046
iteration 600 / 1000: loss 39.739480
iteration 700 / 1000: loss 24.938779
iteration 800 / 1000: loss 15.844841
iteration 900 / 1000: loss 10.465389
iteration 0 / 1000: loss 761.516019
iteration 100 / 1000: loss 460.703963
iteration 200 / 1000: loss 279.032959
iteration 300 / 1000: loss 169.611113
iteration 400 / 1000: loss 103.416886
iteration 500 / 1000: loss 63.382137
iteration 600 / 1000: loss 39.251058
iteration 700 / 1000: loss 24.597912
iteration 800 / 1000: loss 15.660858
iteration 900 / 1000: loss 10.313536
iteration 0 / 1000: loss 781.281315
iteration 100 / 1000: loss 472.354664
iteration 200 / 1000: loss 286.298076
iteration 300 / 1000: loss 173.883577
iteration 400 / 1000: loss 106.115766
iteration 500 / 1000: loss 65.075604
iteration 600 / 1000: loss 40

iteration 0 / 1000: loss 774.071180
iteration 100 / 1000: loss 467.614939
iteration 200 / 1000: loss 283.490615
iteration 300 / 1000: loss 172.364316
iteration 400 / 1000: loss 105.019218
iteration 500 / 1000: loss 64.406040
iteration 600 / 1000: loss 39.811118
iteration 700 / 1000: loss 24.910065
iteration 800 / 1000: loss 15.908347
iteration 900 / 1000: loss 10.378711
iteration 0 / 1000: loss 768.101901
iteration 100 / 1000: loss 464.583607
iteration 200 / 1000: loss 281.503445
iteration 300 / 1000: loss 171.227062
iteration 400 / 1000: loss 104.310060
iteration 500 / 1000: loss 63.991708
iteration 600 / 1000: loss 39.491572
iteration 700 / 1000: loss 24.707287
iteration 800 / 1000: loss 15.841485
iteration 900 / 1000: loss 10.392029
iteration 0 / 1000: loss 776.982400
iteration 100 / 1000: loss 469.278475
iteration 200 / 1000: loss 284.521468
iteration 300 / 1000: loss 172.767199
iteration 400 / 1000: loss 105.233767
iteration 500 / 1000: loss 64.631960
iteration 600 / 1000: loss 39

iteration 0 / 1000: loss 772.508610
iteration 100 / 1000: loss 466.828435
iteration 200 / 1000: loss 283.076804
iteration 300 / 1000: loss 172.060335
iteration 400 / 1000: loss 104.760946
iteration 500 / 1000: loss 64.365471
iteration 600 / 1000: loss 39.669671
iteration 700 / 1000: loss 24.905813
iteration 800 / 1000: loss 15.847382
iteration 900 / 1000: loss 10.457858
iteration 0 / 1000: loss 768.709740
iteration 100 / 1000: loss 464.260245
iteration 200 / 1000: loss 281.482161
iteration 300 / 1000: loss 171.060436
iteration 400 / 1000: loss 104.211775
iteration 500 / 1000: loss 63.923005
iteration 600 / 1000: loss 39.472013
iteration 700 / 1000: loss 24.728573
iteration 800 / 1000: loss 15.825905
iteration 900 / 1000: loss 10.369754
iteration 0 / 1000: loss 759.127797
iteration 100 / 1000: loss 459.218571
iteration 200 / 1000: loss 278.503306
iteration 300 / 1000: loss 169.215955
iteration 400 / 1000: loss 103.259993
iteration 500 / 1000: loss 63.315629
iteration 600 / 1000: loss 39

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question 2** - *True or False*

Suppose the overall training loss is defined as the sum of the per-datapoint loss over all training examples. It is possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

$\color{blue}{\textit Your Answer:}$


$\color{blue}{\textit Your Explanation:}$



In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])